## initial values

In [ ]:
#!/usr/bin/env python
import numpy as np  # use the math functions
from IPython.display import HTML
import copy
import ffmpeg
%matplotlib inline

############ Parameters #############

typeA, typeB = "disk", "disk"
mA, mB = 3e11, 3e10	# mass [in solar mass]
rA, rB = 4.0, 2.0 # scale-radius [in kpc]
axisA, axisB = np.array([0.0,0.0,1.0]), np.array([0.0,0.0,1.0]) # axis

xA, xB = 0.0, 10.0 # initial position [in kpc]
yA, yB = 0.0, -30.0
zA, zB = 0.0, 0.0

vxA, vxB = 0, 0.05  # initial velocity [in kpc/Myr]
vyA, vyB = 0, 0.05
vzA, vzB = 0, 0.0


n = 5000 # total number of tracer particles
t_end = 2000. # final time [in Myr]
size = 100 # size of the plot [in kpc]

figsize = 10
animfreq = 50 # frequency of the frames for the animation (large value = slow)
outputfreq = 0 # frequency of the frames for the output as PNG (0 = off)


# You don't need to change anything below this line (and you shouldn't ...) #
#############################################################################

### functions to initialize disks of tracers ################################

def rotation(x, y, z, axis):
	dat = np.column_stack((x,y,z))
	axis = axis / np.sqrt(axis[0]**2 + axis[1]**2 + axis[2]**2) # normalize the axis

	if(not np.array_equal(axis, np.array([0,0,1]))):
		if(np.array_equal(axis, np.array([0,0,-1]))):
			rot = np.array([1,0,0])
			cangle = -1.0
			sangle = 0.0
		else:
			rot = np.array([-axis[1], axis[0], 0.0])  # rotation axis is the vector product of (z-axis, gal_axis)
			sangle = np.sqrt(axis[0]**2+axis[1]**2) # sine of rotation angle is the norm of vector product (z-axis, gal_axis)
			rot = rot / sangle # normalisation of rotation axis
			cangle = axis[2] # cosine of the rotation angle

		p = np.identity(3)*0.0 # p = rot * transpose(rot)
		q = p*0.0
		p[:,0] = rot * rot[0]
		p[:,1] = rot * rot[1]
		p[:,2] = rot * rot[2]
		q[0,1] = -rot[2]
		q[1,0] =  rot[2]
		q[0,2] =  rot[1]
		q[2,0] = -rot[1]
		q[1,2] = -rot[0]
		q[2,1] =  rot[0]	
		rot_matrix = p + (np.identity(3)-p)*cangle + q*sangle

		for i in range(len(x)):
			dat = np.matmul(rot_matrix, np.array([x[i], y[i], z[i]]))
			x[i], y[i], z[i] = dat[0], dat[1], dat[2]

	return x, y, z

def init_galaxy(type, n, r0, rmin, x0, y0, z0, vx0, vy0, vz0, M, axis=np.array([0,0,1])): # type, number of particles, radius, min radius, initial galaxy position, initial galaxy velocity, mass, spin axis
	G = 4.5e-12 	# [in kpc**3 / solar mass / Myr**2]
	phi = np.random.rand(n) * 2.*np.pi

	if(type == "disk"):		
		rcut = 5.*r0 # arbitrary scaling
		radius = np.zeros(n)
		for i in range(n):
			while True: 
				rad = np.random.rand() * (rcut-rmin) + rmin
				pdf = rad / r0 * np.exp(1.-rad/r0)
				if(np.random.rand() <= pdf):
					radius[i]=rad
					break
		vel = np.sqrt(G*M / radius)
		x = radius * np.cos(phi)
		y = radius * np.sin(phi)
		z = x*0.0
		vx = -1.0 * vel * np.sin(phi)
		vy = vel * np.cos(phi)
		vz = vx*0.0
		x, y, z = rotation(x, y, z, axis)
		vx, vy, vz = rotation(vx, vy, vz, axis)
	else:
		radius = np.random.rand(n) * (r0-rmin) + rmin
		theta = np.random.rand(n) * np.pi
		vel = np.sqrt(G*M / radius) / 100.0 # cheat here
		x = radius * np.cos(phi)*np.cos(theta)
		y = radius * np.sin(phi)*np.cos(theta)
		z = radius * np.sin(theta)
		fx = np.random.rand(n)
		fy = np.random.rand(n)
		vx = vel*fx # quick and dirty random phase
		vy = (1.-fx)*fy * vel
		vz = -(x*vx + y*vy) / z # ensure dot product x . v = 0

	return x+x0, y+y0, z+z0, vx+vx0, vy+vy0, vz+vz0	

############ Initialization #############

nA = np.ceil(n * mA / (mA+mB)).astype(int) # number of tracers in galaxy A
nB = n - nA
x, y, z = np.zeros([n+2]), np.zeros([n+2]), np.zeros([n+2])
vx, vy, vz = np.zeros([n+2]), np.zeros([n+2]), np.zeros([n+2])

time = 0.0 
dt = 0.1 # timestep [in Myr]
eps = 0.3 # force softening [in kpc]

x[0], y[0], z[0] = xA, yA, zA
x[1], y[1], z[1] = xB, yB, zB
vx[0], vy[0], vz[0] = vxA, vyA, vzA
vx[1], vy[1], vz[1] = vxB, vyB, vzB

# initialize the tracers of the 2 galaxies
x[2:nA+2], y[2:nA+2], z[2:nA+2], vx[2:nA+2], vy[2:nA+2], vz[2:nA+2] = init_galaxy(typeA, nA, rA, eps, xA, yA, zA, vxA, vyA, vzA, mA, axisA) 
x[2+nA:], y[2+nA:], z[2+nA:], vx[2+nA:], vy[2+nA:], vz[2+nA:] = init_galaxy(typeB, nB, rB, eps, xB, yB, zB, vxB, vyB, vzB, mB, axisB)

dtout = 50 		# output every dtout timesteps
fout = dtout * outputfreq
pt = np.zeros((np.ceil(t_end/dt/dtout).astype(int),n+2,3)) # array to store the results
step = 0 # counter for output

def acceleration(m, x1, y1, z1, x2, y2, z2):
	Gmr3 = -4.5e-12 * m / ( np.sqrt( (x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2 + eps) )**3 	# [in Myr**-2]
	ax = (x2-x1) * Gmr3
	ay = (y2-y1) * Gmr3
	az = (z2-z1) * Gmr3
	return ax, ay, az

while time < t_end:  ############ Time integration: leap-frog method

	ax_A, ay_A, az_A = acceleration(mA, x[0], y[0], z[0], x, y, z) 	# update acceleration from galaxy A
	ax_A[0], ay_A[0], az_A[0] = 0.0, 0.0, 0.0

	ax_B, ay_B, az_B = acceleration(mB, x[1], y[1], z[1], x, y, z) # update acceleration from galaxy B
	ax_B[1], ay_B[1], az_B[1] = 0.0, 0.0, 0.0

	vx = vx + (ax_A + ax_B) * dt # update velocity
	vy = vy + (ay_A + ay_B) * dt
	vz = vz + (az_A + az_B) * dt

	x = x + vx * dt # update position
	y = y + vy * dt
	z = z + vz * dt

	time = time + dt # update time
	step = step + 1

	if(step % dtout == 0): # output for movie display
		i = np.int(step/dtout-1) 
		pt[i,:,0], pt[i,:,1], pt[i,:,2] = x, y, z


### Movie ########################################################
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig, ax = plt.subplots(figsize=(figsize,figsize))

gA, = ax.plot([], [], 'o', markersize=1.5, color='darkblue')
gB, = ax.plot([], [], 'o', markersize=1.5, color='darkred')
ax.axis('equal')
ax.set_xlim([-size/2.,size/2.])
ax.set_ylim([-size/2.,size/2.])
ax.set_xlabel('x [kpc]')
ax.set_ylabel('y [kpc]')
txt = ax.text(0.4*size,-0.4*size, ' Myr', horizontalalignment='right')

def animate(i):
	gA.set_data(pt[i,2:nA+2,0], pt[i,2:nA+2,1])
	gB.set_data(pt[i,nA+2:,0], pt[i,nA+2:,1])
	txt.set_text('{:.0f} Myr'.format(i*dtout*dt))
	if(fout > 0 and i % fout == 0):
		plt.savefig("f_"+str(i).zfill(5)+".png")
	return gA, gB, txt

nf = int(np.ceil(t_end/dt/dtout))
ani = animation.FuncAnimation(fig, animate, frames=nf, interval=animfreq, blit=True)
#ani.save('movie.mp4')
HTML(ani.to_html5_video())


## Circular pericentre

In [ ]:
#!/usr/bin/env python
import numpy as np  # use the math functions
from IPython.display import HTML
import copy
import ffmpeg
%matplotlib inline

############ Parameters #############

typeA, typeB = "disk", "disk"
mA, mB = 3e11, 3e10	# mass [in solar mass]
rA, rB = 4.0, 4.0 # scale-radius [in kpc]
axisA, axisB = np.array([0.0,0.0,1.0]), np.array([0.0,0.0,1.0]) # axis

xA, xB = 0.0, 17 # initial position [in kpc]
yA, yB = 0.0, 0.0
zA, zB = 0.0, 0.0

vxA, vxB = 0, 0.0 # initial velocity [in kpc/Myr]
vyA, vyB = 0, 0.28
vzA, vzB = 0, 0.0


n = 7000 # total number of tracer particles
t_end = 400. # final time [in Myr]
size = 150 # size of the plot [in kpc]

figsize = 10
animfreq = 50 # frequency of the frames for the animation (large value = slow)
outputfreq = 0.0 # frequency of the frames for the output as PNG (0 = off)


# You don't need to change anything below this line (and you shouldn't ...) #
#############################################################################

### functions to initialize disks of tracers ################################

def rotation(x, y, z, axis):
	dat = np.column_stack((x,y,z))
	axis = axis / np.sqrt(axis[0]**2 + axis[1]**2 + axis[2]**2) # normalize the axis

	if(not np.array_equal(axis, np.array([0,0,1]))):
		if(np.array_equal(axis, np.array([0,0,-1]))):
			rot = np.array([1,0,0])
			cangle = -1.0
			sangle = 0.0
		else:
			rot = np.array([-axis[1], axis[0], 0.0])  # rotation axis is the vector product of (z-axis, gal_axis)
			sangle = np.sqrt(axis[0]**2+axis[1]**2) # sine of rotation angle is the norm of vector product (z-axis, gal_axis)
			rot = rot / sangle # normalisation of rotation axis
			cangle = axis[2] # cosine of the rotation angle

		p = np.identity(3)*0.0 # p = rot * transpose(rot)
		q = p*0.0
		p[:,0] = rot * rot[0]
		p[:,1] = rot * rot[1]
		p[:,2] = rot * rot[2]
		q[0,1] = -rot[2]
		q[1,0] =  rot[2]
		q[0,2] =  rot[1]
		q[2,0] = -rot[1]
		q[1,2] = -rot[0]
		q[2,1] =  rot[0]	
		rot_matrix = p + (np.identity(3)-p)*cangle + q*sangle

		for i in range(len(x)):
			dat = np.matmul(rot_matrix, np.array([x[i], y[i], z[i]]))
			x[i], y[i], z[i] = dat[0], dat[1], dat[2]

	return x, y, z

def init_galaxy(type, n, r0, rmin, x0, y0, z0, vx0, vy0, vz0, M, axis=np.array([0,0,1])): # type, number of particles, radius, min radius, initial galaxy position, initial galaxy velocity, mass, spin axis
	G = 4.5e-12 	# [in kpc**3 / solar mass / Myr**2]
	phi = np.random.rand(n) * 2.*np.pi

	if(type == "disk"):		
		rcut = 5.*r0 # arbitrary scaling
		radius = np.zeros(n)
		for i in range(n):
			while True: 
				rad = np.random.rand() * (rcut-rmin) + rmin
				pdf = rad / r0 * np.exp(1.-rad/r0)
				if(np.random.rand() <= pdf):
					radius[i]=rad
					break
		vel = np.sqrt(G*M / radius)
		x = radius * np.cos(phi)
		y = radius * np.sin(phi)
		z = x*0.0
		vx = -1.0 * vel * np.sin(phi)
		vy = vel * np.cos(phi)
		vz = vx*0.0
		x, y, z = rotation(x, y, z, axis)
		vx, vy, vz = rotation(vx, vy, vz, axis)
	else:
		radius = np.random.rand(n) * (r0-rmin) + rmin
		theta = np.random.rand(n) * np.pi
		vel = np.sqrt(G*M / radius) / 100.0 # cheat here
		x = radius * np.cos(phi)*np.cos(theta)
		y = radius * np.sin(phi)*np.cos(theta)
		z = radius * np.sin(theta)
		fx = np.random.rand(n)
		fy = np.random.rand(n)
		vx = vel*fx # quick and dirty random phase
		vy = (1.-fx)*fy * vel
		vz = -(x*vx + y*vy) / z # ensure dot product x . v = 0

	return x+x0, y+y0, z+z0, vx+vx0, vy+vy0, vz+vz0	

############ Initialization #############

nA = np.ceil(n * mA / (mA+mB)).astype(int) # number of tracers in galaxy A
nB = n - nA
x, y, z = np.zeros([n+2]), np.zeros([n+2]), np.zeros([n+2])
vx, vy, vz = np.zeros([n+2]), np.zeros([n+2]), np.zeros([n+2])

time = 0.0 
dt = 0.1 # timestep [in Myr]
eps = 0.3 # force softening [in kpc]

x[0], y[0], z[0] = xA, yA, zA
x[1], y[1], z[1] = xB, yB, zB
vx[0], vy[0], vz[0] = vxA, vyA, vzA
vx[1], vy[1], vz[1] = vxB, vyB, vzB

# initialize the tracers of the 2 galaxies
x[2:nA+2], y[2:nA+2], z[2:nA+2], vx[2:nA+2], vy[2:nA+2], vz[2:nA+2] = init_galaxy(typeA, nA, rA, eps, xA, yA, zA, vxA, vyA, vzA, mA, axisA) 
x[2+nA:], y[2+nA:], z[2+nA:], vx[2+nA:], vy[2+nA:], vz[2+nA:] = init_galaxy(typeB, nB, rB, eps, xB, yB, zB, vxB, vyB, vzB, mB, axisB)

dtout = 50 		# output every dtout timesteps
fout = dtout * outputfreq
pt = np.zeros((np.ceil(t_end/dt/dtout).astype(int),n+2,3)) # array to store the results
step = 0 # counter for output

def acceleration(m, x1, y1, z1, x2, y2, z2):
	Gmr3 = -4.5e-12 * m / ( np.sqrt( (x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2 + eps) )**3 	# [in Myr**-2]
	ax = (x2-x1) * Gmr3
	ay = (y2-y1) * Gmr3
	az = (z2-z1) * Gmr3
	return ax, ay, az

while time < t_end:  ############ Time integration: leap-frog method

	ax_A, ay_A, az_A = acceleration(mA, x[0], y[0], z[0], x, y, z) 	# update acceleration from galaxy A
	ax_A[0], ay_A[0], az_A[0] = 0.0, 0.0, 0.0

	ax_B, ay_B, az_B = acceleration(mB, x[1], y[1], z[1], x, y, z) # update acceleration from galaxy B
	ax_B[1], ay_B[1], az_B[1] = 0.0, 0.0, 0.0

	vx = vx + (ax_A + ax_B) * dt # update velocity
	vy = vy + (ay_A + ay_B) * dt
	vz = vz + (az_A + az_B) * dt

	x = x + vx * dt # update position
	y = y + vy * dt
	z = z + vz * dt

	time = time + dt # update time
	step = step + 1

	if(step % dtout == 0): # output for movie display
		i = np.int(step/dtout-1) 
		pt[i,:,0], pt[i,:,1], pt[i,:,2] = x, y, z


### Movie ########################################################
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig, ax = plt.subplots(figsize=(figsize,figsize))

gA, = ax.plot([], [], 'o', markersize=1.5, color='darkblue')
gB, = ax.plot([], [], 'o', markersize=1.5, color='darkred')
ax.axis('equal')
ax.set_xlim([-size/2.,size/2.])
ax.set_ylim([-size/2.,size/2.])
ax.set_xlabel('x [kpc]')
ax.set_ylabel('y [kpc]')
txt = ax.text(0.4*size,-0.4*size, ' Myr', horizontalalignment='right')

def animate(i):
	gA.set_data(pt[i,2:nA+2,0], pt[i,2:nA+2,1])
	gB.set_data(pt[i,nA+2:,0], pt[i,nA+2:,1])
	txt.set_text('{:.0f} Myr'.format(i*dtout*dt))
	if(fout > 0 and i % fout == 0):
		plt.savefig("fcircle_"+str(i).zfill(5)+".png")
	return gA, gB, txt

nf = int(np.ceil(t_end/dt/dtout))
ani = animation.FuncAnimation(fig, animate, frames=nf, interval=animfreq, blit=True)
#ani.save('movie.mp4')
HTML(ani.to_html5_video())


## Cahnged ang.mom direction

In [ ]:
#!/usr/bin/env python
import numpy as np  # use the math functions
from IPython.display import HTML
import copy
import ffmpeg
%matplotlib inline

############ Parameters #############

typeA, typeB = "disk", "sphere"
mA, mB = 3e11, 3e10	# mass [in solar mass]
rA, rB = 4.0, 2.0 # scale-radius [in kpc]
axisA, axisB = np.array([0.0,0.0,-1.0]), np.array([0.0,0.0,1.0]) # axis

xA, xB = 0.0, 30.0 # initial position [in kpc]
yA, yB = 0.0, -30.0
zA, zB = 0.0, 0.0

vxA, vxB = 0, 0.1  # initial velocity [in kpc/Myr]
vyA, vyB = 0, 0.1
vzA, vzB = 0, 0.0


n = 5000 # total number of tracer particles
t_end = 2000. # final time [in Myr]
size = 100 # size of the plot [in kpc]

figsize = 10
animfreq = 50 # frequency of the frames for the animation (large value = slow)
outputfreq = 0 # frequency of the frames for the output as PNG (0 = off)


# You don't need to change anything below this line (and you shouldn't ...) #
#############################################################################

### functions to initialize disks of tracers ################################

def rotation(x, y, z, axis):
	dat = np.column_stack((x,y,z))
	axis = axis / np.sqrt(axis[0]**2 + axis[1]**2 + axis[2]**2) # normalize the axis

	if(not np.array_equal(axis, np.array([0,0,1]))):
		if(np.array_equal(axis, np.array([0,0,-1]))):
			rot = np.array([1,0,0])
			cangle = -1.0
			sangle = 0.0
		else:
			rot = np.array([-axis[1], axis[0], 0.0])  # rotation axis is the vector product of (z-axis, gal_axis)
			sangle = np.sqrt(axis[0]**2+axis[1]**2) # sine of rotation angle is the norm of vector product (z-axis, gal_axis)
			rot = rot / sangle # normalisation of rotation axis
			cangle = axis[2] # cosine of the rotation angle

		p = np.identity(3)*0.0 # p = rot * transpose(rot)
		q = p*0.0
		p[:,0] = rot * rot[0]
		p[:,1] = rot * rot[1]
		p[:,2] = rot * rot[2]
		q[0,1] = -rot[2]
		q[1,0] =  rot[2]
		q[0,2] =  rot[1]
		q[2,0] = -rot[1]
		q[1,2] = -rot[0]
		q[2,1] =  rot[0]	
		rot_matrix = p + (np.identity(3)-p)*cangle + q*sangle

		for i in range(len(x)):
			dat = np.matmul(rot_matrix, np.array([x[i], y[i], z[i]]))
			x[i], y[i], z[i] = dat[0], dat[1], dat[2]

	return x, y, z

def init_galaxy(type, n, r0, rmin, x0, y0, z0, vx0, vy0, vz0, M, axis=np.array([0,0,1])): # type, number of particles, radius, min radius, initial galaxy position, initial galaxy velocity, mass, spin axis
	G = 4.5e-12 	# [in kpc**3 / solar mass / Myr**2]
	phi = np.random.rand(n) * 2.*np.pi

	if(type == "disk"):		
		rcut = 5.*r0 # arbitrary scaling
		radius = np.zeros(n)
		for i in range(n):
			while True: 
				rad = np.random.rand() * (rcut-rmin) + rmin
				pdf = rad / r0 * np.exp(1.-rad/r0)
				if(np.random.rand() <= pdf):
					radius[i]=rad
					break
		vel = np.sqrt(G*M / radius)
		x = radius * np.cos(phi)
		y = radius * np.sin(phi)
		z = x*0.0
		vx = -1.0 * vel * np.sin(phi)
		vy = vel * np.cos(phi)
		vz = vx*0.0
		x, y, z = rotation(x, y, z, axis)
		vx, vy, vz = rotation(vx, vy, vz, axis)
	else:
		radius = np.random.rand(n) * (r0-rmin) + rmin
		theta = np.random.rand(n) * np.pi
		vel = np.sqrt(G*M / radius) / 100.0 # cheat here
		x = radius * np.cos(phi)*np.cos(theta)
		y = radius * np.sin(phi)*np.cos(theta)
		z = radius * np.sin(theta)
		fx = np.random.rand(n)
		fy = np.random.rand(n)
		vx = vel*fx # quick and dirty random phase
		vy = (1.-fx)*fy * vel
		vz = -(x*vx + y*vy) / z # ensure dot product x . v = 0

	return x+x0, y+y0, z+z0, vx+vx0, vy+vy0, vz+vz0	

############ Initialization #############

nA = np.ceil(n * mA / (mA+mB)).astype(int) # number of tracers in galaxy A
nB = n - nA
x, y, z = np.zeros([n+2]), np.zeros([n+2]), np.zeros([n+2])
vx, vy, vz = np.zeros([n+2]), np.zeros([n+2]), np.zeros([n+2])

time = 0.0 
dt = 0.1 # timestep [in Myr]
eps = 0.3 # force softening [in kpc]

x[0], y[0], z[0] = xA, yA, zA
x[1], y[1], z[1] = xB, yB, zB
vx[0], vy[0], vz[0] = vxA, vyA, vzA
vx[1], vy[1], vz[1] = vxB, vyB, vzB

# initialize the tracers of the 2 galaxies
x[2:nA+2], y[2:nA+2], z[2:nA+2], vx[2:nA+2], vy[2:nA+2], vz[2:nA+2] = init_galaxy(typeA, nA, rA, eps, xA, yA, zA, vxA, vyA, vzA, mA, axisA) 
x[2+nA:], y[2+nA:], z[2+nA:], vx[2+nA:], vy[2+nA:], vz[2+nA:] = init_galaxy(typeB, nB, rB, eps, xB, yB, zB, vxB, vyB, vzB, mB, axisB)

dtout = 50 		# output every dtout timesteps
fout = dtout * outputfreq
pt = np.zeros((np.ceil(t_end/dt/dtout).astype(int),n+2,3)) # array to store the results
step = 0 # counter for output

def acceleration(m, x1, y1, z1, x2, y2, z2):
	Gmr3 = -4.5e-12 * m / ( np.sqrt( (x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2 + eps) )**3 	# [in Myr**-2]
	ax = (x2-x1) * Gmr3
	ay = (y2-y1) * Gmr3
	az = (z2-z1) * Gmr3
	return ax, ay, az

while time < t_end:  ############ Time integration: leap-frog method

	ax_A, ay_A, az_A = acceleration(mA, x[0], y[0], z[0], x, y, z) 	# update acceleration from galaxy A
	ax_A[0], ay_A[0], az_A[0] = 0.0, 0.0, 0.0

	ax_B, ay_B, az_B = acceleration(mB, x[1], y[1], z[1], x, y, z) # update acceleration from galaxy B
	ax_B[1], ay_B[1], az_B[1] = 0.0, 0.0, 0.0

	vx = vx + (ax_A + ax_B) * dt # update velocity
	vy = vy + (ay_A + ay_B) * dt
	vz = vz + (az_A + az_B) * dt

	x = x + vx * dt # update position
	y = y + vy * dt
	z = z + vz * dt

	time = time + dt # update time
	step = step + 1

	if(step % dtout == 0): # output for movie display
		i = np.int(step/dtout-1) 
		pt[i,:,0], pt[i,:,1], pt[i,:,2] = x, y, z


### Movie ########################################################
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig, ax = plt.subplots(figsize=(figsize,figsize))

gA, = ax.plot([], [], 'o', markersize=1.5, color='darkblue')
gB, = ax.plot([], [], 'o', markersize=1.5, color='darkred')
ax.axis('equal')
ax.set_xlim([-size/2.,size/2.])
ax.set_ylim([-size/2.,size/2.])
ax.set_xlabel('x [kpc]')
ax.set_ylabel('y [kpc]')
txt = ax.text(0.4*size,-0.4*size, ' Myr', horizontalalignment='right')

def animate(i):
	gA.set_data(pt[i,2:nA+2,0], pt[i,2:nA+2,1])
	gB.set_data(pt[i,nA+2:,0], pt[i,nA+2:,1])
	txt.set_text('{:.0f} Myr'.format(i*dtout*dt))
	if(fout > 0 and i % fout == 0):
		plt.savefig("f_"+str(i).zfill(5)+".png")
	return gA, gB, txt

nf = int(np.ceil(t_end/dt/dtout))
ani = animation.FuncAnimation(fig, animate, frames=nf, interval=animfreq, blit=True)
#ani.save('movie.mp4')
HTML(ani.to_html5_video())


## similar models

In [ ]:
#!/usr/bin/env python
import numpy as np  # use the math functions
from IPython.display import HTML
import copy
import ffmpeg
%matplotlib inline

############ Parameters #############

typeA, typeB = "disk", "sphere"
mA, mB = 3e11, 3e11	# mass [in solar mass]
rA, rB = 4.0, 2.0 # scale-radius [in kpc]
axisA, axisB = np.array([0.0,0.0,1.0]), np.array([0.0,0.0,1.0]) # axis

xA, xB = 0.0, 30.0 # initial position [in kpc]
yA, yB = 0.0, -30.0
zA, zB = 0.0, 0.0

vxA, vxB = 0, 0.1  # initial velocity [in kpc/Myr]
vyA, vyB = 0, 0.1
vzA, vzB = 0, 0.0


n = 5000 # total number of tracer particles
t_end = 4000. # final time [in Myr]
size = 400 # size of the plot [in kpc]

figsize = 10
animfreq = 50 # frequency of the frames for the animation (large value = slow)
outputfreq = 0 # frequency of the frames for the output as PNG (0 = off)


# You don't need to change anything below this line (and you shouldn't ...) #
#############################################################################

### functions to initialize disks of tracers ################################

def rotation(x, y, z, axis):
	dat = np.column_stack((x,y,z))
	axis = axis / np.sqrt(axis[0]**2 + axis[1]**2 + axis[2]**2) # normalize the axis

	if(not np.array_equal(axis, np.array([0,0,1]))):
		if(np.array_equal(axis, np.array([0,0,-1]))):
			rot = np.array([1,0,0])
			cangle = -1.0
			sangle = 0.0
		else:
			rot = np.array([-axis[1], axis[0], 0.0])  # rotation axis is the vector product of (z-axis, gal_axis)
			sangle = np.sqrt(axis[0]**2+axis[1]**2) # sine of rotation angle is the norm of vector product (z-axis, gal_axis)
			rot = rot / sangle # normalisation of rotation axis
			cangle = axis[2] # cosine of the rotation angle

		p = np.identity(3)*0.0 # p = rot * transpose(rot)
		q = p*0.0
		p[:,0] = rot * rot[0]
		p[:,1] = rot * rot[1]
		p[:,2] = rot * rot[2]
		q[0,1] = -rot[2]
		q[1,0] =  rot[2]
		q[0,2] =  rot[1]
		q[2,0] = -rot[1]
		q[1,2] = -rot[0]
		q[2,1] =  rot[0]	
		rot_matrix = p + (np.identity(3)-p)*cangle + q*sangle

		for i in range(len(x)):
			dat = np.matmul(rot_matrix, np.array([x[i], y[i], z[i]]))
			x[i], y[i], z[i] = dat[0], dat[1], dat[2]

	return x, y, z

def init_galaxy(type, n, r0, rmin, x0, y0, z0, vx0, vy0, vz0, M, axis=np.array([0,0,1])): # type, number of particles, radius, min radius, initial galaxy position, initial galaxy velocity, mass, spin axis
	G = 4.5e-12 	# [in kpc**3 / solar mass / Myr**2]
	phi = np.random.rand(n) * 2.*np.pi

	if(type == "disk"):		
		rcut = 5.*r0 # arbitrary scaling
		radius = np.zeros(n)
		for i in range(n):
			while True: 
				rad = np.random.rand() * (rcut-rmin) + rmin
				pdf = rad / r0 * np.exp(1.-rad/r0)
				if(np.random.rand() <= pdf):
					radius[i]=rad
					break
		vel = np.sqrt(G*M / radius)
		x = radius * np.cos(phi)
		y = radius * np.sin(phi)
		z = x*0.0
		vx = -1.0 * vel * np.sin(phi)
		vy = vel * np.cos(phi)
		vz = vx*0.0
		x, y, z = rotation(x, y, z, axis)
		vx, vy, vz = rotation(vx, vy, vz, axis)
	else:
		radius = np.random.rand(n) * (r0-rmin) + rmin
		theta = np.random.rand(n) * np.pi
		vel = np.sqrt(G*M / radius) / 100.0 # cheat here
		x = radius * np.cos(phi)*np.cos(theta)
		y = radius * np.sin(phi)*np.cos(theta)
		z = radius * np.sin(theta)
		fx = np.random.rand(n)
		fy = np.random.rand(n)
		vx = vel*fx # quick and dirty random phase
		vy = (1.-fx)*fy * vel
		vz = -(x*vx + y*vy) / z # ensure dot product x . v = 0

	return x+x0, y+y0, z+z0, vx+vx0, vy+vy0, vz+vz0	

############ Initialization #############

nA = np.ceil(n * mA / (mA+mB)).astype(int) # number of tracers in galaxy A
nB = n - nA
x, y, z = np.zeros([n+2]), np.zeros([n+2]), np.zeros([n+2])
vx, vy, vz = np.zeros([n+2]), np.zeros([n+2]), np.zeros([n+2])

time = 0.0 
dt = 0.1 # timestep [in Myr]
eps = 0.3 # force softening [in kpc]

x[0], y[0], z[0] = xA, yA, zA
x[1], y[1], z[1] = xB, yB, zB
vx[0], vy[0], vz[0] = vxA, vyA, vzA
vx[1], vy[1], vz[1] = vxB, vyB, vzB

# initialize the tracers of the 2 galaxies
x[2:nA+2], y[2:nA+2], z[2:nA+2], vx[2:nA+2], vy[2:nA+2], vz[2:nA+2] = init_galaxy(typeA, nA, rA, eps, xA, yA, zA, vxA, vyA, vzA, mA, axisA) 
x[2+nA:], y[2+nA:], z[2+nA:], vx[2+nA:], vy[2+nA:], vz[2+nA:] = init_galaxy(typeB, nB, rB, eps, xB, yB, zB, vxB, vyB, vzB, mB, axisB)

dtout = 50 		# output every dtout timesteps
fout = dtout * outputfreq
pt = np.zeros((np.ceil(t_end/dt/dtout).astype(int),n+2,3)) # array to store the results
step = 0 # counter for output

def acceleration(m, x1, y1, z1, x2, y2, z2):
	Gmr3 = -4.5e-12 * m / ( np.sqrt( (x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2 + eps) )**3 	# [in Myr**-2]
	ax = (x2-x1) * Gmr3
	ay = (y2-y1) * Gmr3
	az = (z2-z1) * Gmr3
	return ax, ay, az

while time < t_end:  ############ Time integration: leap-frog method

	ax_A, ay_A, az_A = acceleration(mA, x[0], y[0], z[0], x, y, z) 	# update acceleration from galaxy A
	ax_A[0], ay_A[0], az_A[0] = 0.0, 0.0, 0.0

	ax_B, ay_B, az_B = acceleration(mB, x[1], y[1], z[1], x, y, z) # update acceleration from galaxy B
	ax_B[1], ay_B[1], az_B[1] = 0.0, 0.0, 0.0

	vx = vx + (ax_A + ax_B) * dt # update velocity
	vy = vy + (ay_A + ay_B) * dt
	vz = vz + (az_A + az_B) * dt

	x = x + vx * dt # update position
	y = y + vy * dt
	z = z + vz * dt

	time = time + dt # update time
	step = step + 1

	if(step % dtout == 0): # output for movie display
		i = np.int(step/dtout-1) 
		pt[i,:,0], pt[i,:,1], pt[i,:,2] = x, y, z


### Movie ########################################################
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig, ax = plt.subplots(figsize=(figsize,figsize))

gA, = ax.plot([], [], 'o', markersize=1.5, color='darkblue')
gB, = ax.plot([], [], 'o', markersize=1.5, color='darkred')
ax.axis('equal')
ax.set_xlim([-size/2.,size/2.])
ax.set_ylim([-size/2.,size/2.])
ax.set_xlabel('x [kpc]')
ax.set_ylabel('y [kpc]')
txt = ax.text(0.4*size,-0.4*size, ' Myr', horizontalalignment='right')

def animate(i):
	gA.set_data(pt[i,2:nA+2,0], pt[i,2:nA+2,1])
	gB.set_data(pt[i,nA+2:,0], pt[i,nA+2:,1])
	txt.set_text('{:.0f} Myr'.format(i*dtout*dt))
	if(fout > 0 and i % fout == 0):
		plt.savefig("f_"+str(i).zfill(5)+".png")
	return gA, gB, txt

nf = int(np.ceil(t_end/dt/dtout))
ani = animation.FuncAnimation(fig, animate, frames=nf, interval=animfreq, blit=True)
#ani.save('movie.mp4')
HTML(ani.to_html5_video())


## Merging models

In [ ]:
#!/usr/bin/env python
import numpy as np  # use the math functions
from IPython.display import HTML
import copy
import ffmpeg
%matplotlib inline

############ Parameters #############

typeA, typeB = "disk", "disk"
mA, mB = 3e11, 3e10	# mass [in solar mass]
rA, rB = 4.0, 2.0 # scale-radius [in kpc]
axisA, axisB = np.array([0.0,0.0,1.0]), np.array([0.0,0.0,1.0]) # axis

xA, xB = 0.0, 10 # initial position [in kpc]
yA, yB = 0.0, 0.0
zA, zB = 0.0, 0.0

vxA, vxB = 0.0, 0.05 # initial velocity [in kpc/Myr]
vyA, vyB = 0.0, 0.05
vzA, vzB = 0, 0.0


n = 5000 # total number of tracer particles
t_end = 3000. # final time [in Myr]
size = 100 # size of the plot [in kpc]

figsize = 10
animfreq = 50 # frequency of the frames for the animation (large value = slow)
outputfreq = 0.2 # frequency of the frames for the output as PNG (0 = off)


# You don't need to change anything below this line (and you shouldn't ...) #
#############################################################################

### functions to initialize disks of tracers ################################

def rotation(x, y, z, axis):
	dat = np.column_stack((x,y,z))
	axis = axis / np.sqrt(axis[0]**2 + axis[1]**2 + axis[2]**2) # normalize the axis

	if(not np.array_equal(axis, np.array([0,0,1]))):
		if(np.array_equal(axis, np.array([0,0,-1]))):
			rot = np.array([1,0,0])
			cangle = -1.0
			sangle = 0.0
		else:
			rot = np.array([-axis[1], axis[0], 0.0])  # rotation axis is the vector product of (z-axis, gal_axis)
			sangle = np.sqrt(axis[0]**2+axis[1]**2) # sine of rotation angle is the norm of vector product (z-axis, gal_axis)
			rot = rot / sangle # normalisation of rotation axis
			cangle = axis[2] # cosine of the rotation angle

		p = np.identity(3)*0.0 # p = rot * transpose(rot)
		q = p*0.0
		p[:,0] = rot * rot[0]
		p[:,1] = rot * rot[1]
		p[:,2] = rot * rot[2]
		q[0,1] = -rot[2]
		q[1,0] =  rot[2]
		q[0,2] =  rot[1]
		q[2,0] = -rot[1]
		q[1,2] = -rot[0]
		q[2,1] =  rot[0]	
		rot_matrix = p + (np.identity(3)-p)*cangle + q*sangle

		for i in range(len(x)):
			dat = np.matmul(rot_matrix, np.array([x[i], y[i], z[i]]))
			x[i], y[i], z[i] = dat[0], dat[1], dat[2]

	return x, y, z

def init_galaxy(type, n, r0, rmin, x0, y0, z0, vx0, vy0, vz0, M, axis=np.array([0,0,1])): # type, number of particles, radius, min radius, initial galaxy position, initial galaxy velocity, mass, spin axis
	G = 4.5e-12 	# [in kpc**3 / solar mass / Myr**2]
	phi = np.random.rand(n) * 2.*np.pi

	if(type == "disk"):		
		rcut = 5.*r0 # arbitrary scaling
		radius = np.zeros(n)
		for i in range(n):
			while True: 
				rad = np.random.rand() * (rcut-rmin) + rmin
				pdf = rad / r0 * np.exp(1.-rad/r0)
				if(np.random.rand() <= pdf):
					radius[i]=rad
					break
		vel = np.sqrt(G*M / radius)
		x = radius * np.cos(phi)
		y = radius * np.sin(phi)
		z = x*0.0
		vx = -1.0 * vel * np.sin(phi)
		vy = vel * np.cos(phi)
		vz = vx*0.0
		x, y, z = rotation(x, y, z, axis)
		vx, vy, vz = rotation(vx, vy, vz, axis)
	else:
		radius = np.random.rand(n) * (r0-rmin) + rmin
		theta = np.random.rand(n) * np.pi
		vel = np.sqrt(G*M / radius) / 100.0 # cheat here
		x = radius * np.cos(phi)*np.cos(theta)
		y = radius * np.sin(phi)*np.cos(theta)
		z = radius * np.sin(theta)
		fx = np.random.rand(n)
		fy = np.random.rand(n)
		vx = vel*fx # quick and dirty random phase
		vy = (1.-fx)*fy * vel
		vz = -(x*vx + y*vy) / z # ensure dot product x . v = 0

	return x+x0, y+y0, z+z0, vx+vx0, vy+vy0, vz+vz0	

############ Initialization #############

nA = np.ceil(n * mA / (mA+mB)).astype(int) # number of tracers in galaxy A
nB = n - nA
x, y, z = np.zeros([n+2]), np.zeros([n+2]), np.zeros([n+2])
vx, vy, vz = np.zeros([n+2]), np.zeros([n+2]), np.zeros([n+2])

time = 0.0 
dt = 0.1 # timestep [in Myr]
eps = 0.3 # force softening [in kpc]

x[0], y[0], z[0] = xA, yA, zA
x[1], y[1], z[1] = xB, yB, zB
vx[0], vy[0], vz[0] = vxA, vyA, vzA
vx[1], vy[1], vz[1] = vxB, vyB, vzB

# initialize the tracers of the 2 galaxies
x[2:nA+2], y[2:nA+2], z[2:nA+2], vx[2:nA+2], vy[2:nA+2], vz[2:nA+2] = init_galaxy(typeA, nA, rA, eps, xA, yA, zA, vxA, vyA, vzA, mA, axisA) 
x[2+nA:], y[2+nA:], z[2+nA:], vx[2+nA:], vy[2+nA:], vz[2+nA:] = init_galaxy(typeB, nB, rB, eps, xB, yB, zB, vxB, vyB, vzB, mB, axisB)

dtout = 50 		# output every dtout timesteps
fout = dtout * outputfreq
pt = np.zeros((np.ceil(t_end/dt/dtout).astype(int),n+2,3)) # array to store the results
step = 0 # counter for output

def acceleration(m, x1, y1, z1, x2, y2, z2):
	Gmr3 = -4.5e-12 * m / ( np.sqrt( (x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2 + eps) )**3 	# [in Myr**-2]
	ax = (x2-x1) * Gmr3
	ay = (y2-y1) * Gmr3
	az = (z2-z1) * Gmr3
	return ax, ay, az

while time < t_end:  ############ Time integration: leap-frog method

	ax_A, ay_A, az_A = acceleration(mA, x[0], y[0], z[0], x, y, z) 	# update acceleration from galaxy A
	ax_A[0], ay_A[0], az_A[0] = 0.0, 0.0, 0.0

	ax_B, ay_B, az_B = acceleration(mB, x[1], y[1], z[1], x, y, z) # update acceleration from galaxy B
	ax_B[1], ay_B[1], az_B[1] = 0.0, 0.0, 0.0

	vx = vx + (ax_A + ax_B) * dt # update velocity
	vy = vy + (ay_A + ay_B) * dt
	vz = vz + (az_A + az_B) * dt

	x = x + vx * dt # update position
	y = y + vy * dt
	z = z + vz * dt

	time = time + dt # update time
	step = step + 1

	if(step % dtout == 0): # output for movie display
		i = np.int(step/dtout-1) 
		pt[i,:,0], pt[i,:,1], pt[i,:,2] = x, y, z


### Movie ########################################################
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig, ax = plt.subplots(figsize=(figsize,figsize))

gA, = ax.plot([], [], 'o', markersize=1.5, color='darkblue')
gB, = ax.plot([], [], 'o', markersize=1.5, color='darkred')
ax.axis('equal')
ax.set_xlim([-size/2.,size/2.])
ax.set_ylim([-size/2.,size/2.])
ax.set_xlabel('x [kpc]')
ax.set_ylabel('y [kpc]')
txt = ax.text(0.4*size,-0.4*size, ' Myr', horizontalalignment='right')

def animate(i):
	gA.set_data(pt[i,2:nA+2,0], pt[i,2:nA+2,1])
	gB.set_data(pt[i,nA+2:,0], pt[i,nA+2:,1])
	txt.set_text('{:.0f} Myr'.format(i*dtout*dt))
	if(fout > 0 and i % fout == 0):
		plt.savefig("fother_"+str(i).zfill(5)+".png")
	return gA, gB, txt

nf = int(np.ceil(t_end/dt/dtout))
ani = animation.FuncAnimation(fig, animate, frames=nf, interval=animfreq, blit=True)
#ani.save('movie.mp4')
HTML(ani.to_html5_video())


## Ring galaxy

In [ ]:
#!/usr/bin/env python
import numpy as np  # use the math functions
from IPython.display import HTML
import copy
import ffmpeg
%matplotlib inline

############ Parameters #############

typeA, typeB = "disk", "sphere"
mA, mB = 3e11, 9e10	# mass [in solar mass]
rA, rB = 4.0, 2.0 # scale-radius [in kpc]
axisA, axisB = np.array([0.0,0.0,1.0]), np.array([0.0,0.0,1.0]) # axis

xA, xB = 0.0, 0.0 # initial position [in kpc]
yA, yB = 0.0, 0
zA, zB = 0.0, -15.0

vxA, vxB = 0.0, 0.0 # initial velocity [in kpc/Myr]
vyA, vyB = 0.0, 0.0
vzA, vzB = 0, 0.05


n = 5000 # total number of tracer particles
t_end = 2000. # final time [in Myr]
size = 100 # size of the plot [in kpc]

figsize = 10
animfreq = 50 # frequency of the frames for the animation (large value = slow)
outputfreq = 0.0 # frequency of the frames for the output as PNG (0 = off)


# You don't need to change anything below this line (and you shouldn't ...) #
#############################################################################

### functions to initialize disks of tracers ################################

def rotation(x, y, z, axis):
	dat = np.column_stack((x,y,z))
	axis = axis / np.sqrt(axis[0]**2 + axis[1]**2 + axis[2]**2) # normalize the axis

	if(not np.array_equal(axis, np.array([0,0,1]))):
		if(np.array_equal(axis, np.array([0,0,-1]))):
			rot = np.array([1,0,0])
			cangle = -1.0
			sangle = 0.0
		else:
			rot = np.array([-axis[1], axis[0], 0.0])  # rotation axis is the vector product of (z-axis, gal_axis)
			sangle = np.sqrt(axis[0]**2+axis[1]**2) # sine of rotation angle is the norm of vector product (z-axis, gal_axis)
			rot = rot / sangle # normalisation of rotation axis
			cangle = axis[2] # cosine of the rotation angle

		p = np.identity(3)*0.0 # p = rot * transpose(rot)
		q = p*0.0
		p[:,0] = rot * rot[0]
		p[:,1] = rot * rot[1]
		p[:,2] = rot * rot[2]
		q[0,1] = -rot[2]
		q[1,0] =  rot[2]
		q[0,2] =  rot[1]
		q[2,0] = -rot[1]
		q[1,2] = -rot[0]
		q[2,1] =  rot[0]	
		rot_matrix = p + (np.identity(3)-p)*cangle + q*sangle

		for i in range(len(x)):
			dat = np.matmul(rot_matrix, np.array([x[i], y[i], z[i]]))
			x[i], y[i], z[i] = dat[0], dat[1], dat[2]

	return x, y, z

def init_galaxy(type, n, r0, rmin, x0, y0, z0, vx0, vy0, vz0, M, axis=np.array([0,0,1])): # type, number of particles, radius, min radius, initial galaxy position, initial galaxy velocity, mass, spin axis
	G = 4.5e-12 	# [in kpc**3 / solar mass / Myr**2]
	phi = np.random.rand(n) * 2.*np.pi

	if(type == "disk"):		
		rcut = 5.*r0 # arbitrary scaling
		radius = np.zeros(n)
		for i in range(n):
			while True: 
				rad = np.random.rand() * (rcut-rmin) + rmin
				pdf = rad / r0 * np.exp(1.-rad/r0)
				if(np.random.rand() <= pdf):
					radius[i]=rad
					break
		vel = np.sqrt(G*M / radius)
		x = radius * np.cos(phi)
		y = radius * np.sin(phi)
		z = x*0.0
		vx = -1.0 * vel * np.sin(phi)
		vy = vel * np.cos(phi)
		vz = vx*0.0
		x, y, z = rotation(x, y, z, axis)
		vx, vy, vz = rotation(vx, vy, vz, axis)
	else:
		radius = np.random.rand(n) * (r0-rmin) + rmin
		theta = np.random.rand(n) * np.pi
		vel = np.sqrt(G*M / radius) / 100.0 # cheat here
		x = radius * np.cos(phi)*np.cos(theta)
		y = radius * np.sin(phi)*np.cos(theta)
		z = radius * np.sin(theta)
		fx = np.random.rand(n)
		fy = np.random.rand(n)
		vx = vel*fx # quick and dirty random phase
		vy = (1.-fx)*fy * vel
		vz = -(x*vx + y*vy) / z # ensure dot product x . v = 0

	return x+x0, y+y0, z+z0, vx+vx0, vy+vy0, vz+vz0	

############ Initialization #############

nA = np.ceil(n * mA / (mA+mB)).astype(int) # number of tracers in galaxy A
nB = n - nA
x, y, z = np.zeros([n+2]), np.zeros([n+2]), np.zeros([n+2])
vx, vy, vz = np.zeros([n+2]), np.zeros([n+2]), np.zeros([n+2])

time = 0.0 
dt = 0.1 # timestep [in Myr]
eps = 0.3 # force softening [in kpc]

x[0], y[0], z[0] = xA, yA, zA
x[1], y[1], z[1] = xB, yB, zB
vx[0], vy[0], vz[0] = vxA, vyA, vzA
vx[1], vy[1], vz[1] = vxB, vyB, vzB

# initialize the tracers of the 2 galaxies
x[2:nA+2], y[2:nA+2], z[2:nA+2], vx[2:nA+2], vy[2:nA+2], vz[2:nA+2] = init_galaxy(typeA, nA, rA, eps, xA, yA, zA, vxA, vyA, vzA, mA, axisA) 
x[2+nA:], y[2+nA:], z[2+nA:], vx[2+nA:], vy[2+nA:], vz[2+nA:] = init_galaxy(typeB, nB, rB, eps, xB, yB, zB, vxB, vyB, vzB, mB, axisB)

dtout = 50 		# output every dtout timesteps
fout = dtout * outputfreq
pt = np.zeros((np.ceil(t_end/dt/dtout).astype(int),n+2,3)) # array to store the results
step = 0 # counter for output

def acceleration(m, x1, y1, z1, x2, y2, z2):
	Gmr3 = -4.5e-12 * m / ( np.sqrt( (x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2 + eps) )**3 	# [in Myr**-2]
	ax = (x2-x1) * Gmr3
	ay = (y2-y1) * Gmr3
	az = (z2-z1) * Gmr3
	return ax, ay, az

while time < t_end:  ############ Time integration: leap-frog method

	ax_A, ay_A, az_A = acceleration(mA, x[0], y[0], z[0], x, y, z) 	# update acceleration from galaxy A
	ax_A[0], ay_A[0], az_A[0] = 0.0, 0.0, 0.0

	ax_B, ay_B, az_B = acceleration(mB, x[1], y[1], z[1], x, y, z) # update acceleration from galaxy B
	ax_B[1], ay_B[1], az_B[1] = 0.0, 0.0, 0.0

	vx = vx + (ax_A + ax_B) * dt # update velocity
	vy = vy + (ay_A + ay_B) * dt
	vz = vz + (az_A + az_B) * dt

	x = x + vx * dt # update position
	y = y + vy * dt
	z = z + vz * dt

	time = time + dt # update time
	step = step + 1

	if(step % dtout == 0): # output for movie display
		i = np.int(step/dtout-1) 
		pt[i,:,0], pt[i,:,1], pt[i,:,2] = x, y, z


### Movie ########################################################
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig, ax = plt.subplots(figsize=(figsize,figsize))

gA, = ax.plot([], [], 'o', markersize=1.5, color='darkblue')
gB, = ax.plot([], [], 'o', markersize=1.5, color='darkred')
ax.axis('equal')
ax.set_xlim([-size/2.,size/2.])
ax.set_ylim([-size/2.,size/2.])
ax.set_xlabel('x [kpc]')
ax.set_ylabel('y [kpc]')
txt = ax.text(0.4*size,-0.4*size, ' Myr', horizontalalignment='right')

def animate(i):
	gA.set_data(pt[i,2:nA+2,0], pt[i,2:nA+2,1])
	gB.set_data(pt[i,nA+2:,0], pt[i,nA+2:,1])
	txt.set_text('{:.0f} Myr'.format(i*dtout*dt))
	if(fout > 0 and i % fout == 0):
		plt.savefig("fring_"+str(i).zfill(5)+".png")
	return gA, gB, txt

nf = int(np.ceil(t_end/dt/dtout))
ani = animation.FuncAnimation(fig, animate, frames=nf, interval=animfreq, blit=True)
#ani.save('movie.mp4')
HTML(ani.to_html5_video())


## Real Galaxy

In [ ]:
#!/usr/bin/env python
import numpy as np  # use the math functions
from IPython.display import HTML
import copy
import ffmpeg
%matplotlib inline

############ Parameters #############

typeA, typeB = "disk", "disk"
mA, mB = 3e11, 3.3e11	# mass [in solar mass]
rA, rB = 5, 3 # scale-radius [in kpc]
axisA, axisB = np.array([-0.7,2.5,1]), np.array([0.,1.5,0.7]) # axis

xA, xB = -9.0, 5.0 # initial position [in kpc]
yA, yB = 5.0, -30.0
zA, zB = 0.0, 0.0

vxA, vxB = -0.06, 0.12  # initial velocity [in kpc/Myr]
vyA, vyB = -0.0, 0.12
vzA, vzB = 0.0, 0.0


n = 7000 # total number of tracer particles
t_end = 350. # final time [in Myr]
size = 90 # size of the plot [in kpc]

figsize = 10
animfreq = 50 # frequency of the frames for the animation (large value = slow)
outputfreq = 0.0 # frequency of the frames for the output as PNG (0 = off)


# You don't need to change anything below this line (and you shouldn't ...) #
#############################################################################

### functions to initialize disks of tracers ################################

def rotation(x, y, z, axis):
	dat = np.column_stack((x,y,z))
	axis = axis / np.sqrt(axis[0]**2 + axis[1]**2 + axis[2]**2) # normalize the axis

	if(not np.array_equal(axis, np.array([0,0,1]))):
		if(np.array_equal(axis, np.array([0,0,-1]))):
			rot = np.array([1,0,0])
			cangle = -1.0
			sangle = 0.0
		else:
			rot = np.array([-axis[1], axis[0], 0.0])  # rotation axis is the vector product of (z-axis, gal_axis)
			sangle = np.sqrt(axis[0]**2+axis[1]**2) # sine of rotation angle is the norm of vector product (z-axis, gal_axis)
			rot = rot / sangle # normalisation of rotation axis
			cangle = axis[2] # cosine of the rotation angle

		p = np.identity(3)*0.0 # p = rot * transpose(rot)
		q = p*0.0
		p[:,0] = rot * rot[0]
		p[:,1] = rot * rot[1]
		p[:,2] = rot * rot[2]
		q[0,1] = -rot[2]
		q[1,0] =  rot[2]
		q[0,2] =  rot[1]
		q[2,0] = -rot[1]
		q[1,2] = -rot[0]
		q[2,1] =  rot[0]	
		rot_matrix = p + (np.identity(3)-p)*cangle + q*sangle

		for i in range(len(x)):
			dat = np.matmul(rot_matrix, np.array([x[i], y[i], z[i]]))
			x[i], y[i], z[i] = dat[0], dat[1], dat[2]

	return x, y, z

def init_galaxy(type, n, r0, rmin, x0, y0, z0, vx0, vy0, vz0, M, axis=np.array([0,0,1])): # type, number of particles, radius, min radius, initial galaxy position, initial galaxy velocity, mass, spin axis
	G = 4.5e-12 	# [in kpc**3 / solar mass / Myr**2]
	phi = np.random.rand(n) * 2.*np.pi

	if(type == "disk"):		
		rcut = 5.*r0 # arbitrary scaling
		radius = np.zeros(n)
		for i in range(n):
			while True: 
				rad = np.random.rand() * (rcut-rmin) + rmin
				pdf = rad / r0 * np.exp(1.-rad/r0)
				if(np.random.rand() <= pdf):
					radius[i]=rad
					break
		vel = np.sqrt(G*M / radius)
		x = radius * np.cos(phi)
		y = radius * np.sin(phi)
		z = x*0.0
		vx = -1.0 * vel * np.sin(phi)
		vy = vel * np.cos(phi)
		vz = vx*0.0
		x, y, z = rotation(x, y, z, axis)
		vx, vy, vz = rotation(vx, vy, vz, axis)
	else:
		radius = np.random.rand(n) * (r0-rmin) + rmin
		theta = np.random.rand(n) * np.pi
		vel = np.sqrt(G*M / radius) / 100.0 # cheat here
		x = radius * np.cos(phi)*np.cos(theta)
		y = radius * np.sin(phi)*np.cos(theta)
		z = radius * np.sin(theta)
		fx = np.random.rand(n)
		fy = np.random.rand(n)
		vx = vel*fx # quick and dirty random phase
		vy = (1.-fx)*fy * vel
		vz = -(x*vx + y*vy) / z # ensure dot product x . v = 0

	return x+x0, y+y0, z+z0, vx+vx0, vy+vy0, vz+vz0	

############ Initialization #############

nA = np.ceil(n * mA / (mA+mB)).astype(int) # number of tracers in galaxy A
nB = n - nA
x, y, z = np.zeros([n+2]), np.zeros([n+2]), np.zeros([n+2])
vx, vy, vz = np.zeros([n+2]), np.zeros([n+2]), np.zeros([n+2])

time = 0.0 
dt = 0.1 # timestep [in Myr]
eps = 0.3 # force softening [in kpc]

x[0], y[0], z[0] = xA, yA, zA
x[1], y[1], z[1] = xB, yB, zB
vx[0], vy[0], vz[0] = vxA, vyA, vzA
vx[1], vy[1], vz[1] = vxB, vyB, vzB

# initialize the tracers of the 2 galaxies
x[2:nA+2], y[2:nA+2], z[2:nA+2], vx[2:nA+2], vy[2:nA+2], vz[2:nA+2] = init_galaxy(typeA, nA, rA, eps, xA, yA, zA, vxA, vyA, vzA, mA, axisA) 
x[2+nA:], y[2+nA:], z[2+nA:], vx[2+nA:], vy[2+nA:], vz[2+nA:] = init_galaxy(typeB, nB, rB, eps, xB, yB, zB, vxB, vyB, vzB, mB, axisB)

dtout = 50 		# output every dtout timesteps
fout = dtout * outputfreq
pt = np.zeros((np.ceil(t_end/dt/dtout).astype(int),n+2,3)) # array to store the results
step = 0 # counter for output

def acceleration(m, x1, y1, z1, x2, y2, z2):
	Gmr3 = -4.5e-12 * m / ( np.sqrt( (x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2 + eps) )**3 	# [in Myr**-2]
	ax = (x2-x1) * Gmr3
	ay = (y2-y1) * Gmr3
	az = (z2-z1) * Gmr3
	return ax, ay, az

while time < t_end:  ############ Time integration: leap-frog method

	ax_A, ay_A, az_A = acceleration(mA, x[0], y[0], z[0], x, y, z) 	# update acceleration from galaxy A
	ax_A[0], ay_A[0], az_A[0] = 0.0, 0.0, 0.0

	ax_B, ay_B, az_B = acceleration(mB, x[1], y[1], z[1], x, y, z) # update acceleration from galaxy B
	ax_B[1], ay_B[1], az_B[1] = 0.0, 0.0, 0.0

	vx = vx + (ax_A + ax_B) * dt # update velocity
	vy = vy + (ay_A + ay_B) * dt
	vz = vz + (az_A + az_B) * dt

	x = x + vx * dt # update position
	y = y + vy * dt
	z = z + vz * dt

	time = time + dt # update time
	step = step + 1

	if(step % dtout == 0): # output for movie display
		i = np.int(step/dtout-1) 
		pt[i,:,0], pt[i,:,1], pt[i,:,2] = x, y, z


### Movie ########################################################
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig, ax = plt.subplots(figsize=(figsize,figsize))

gA, = ax.plot([], [], 'o', markersize=1.5, color='darkblue')
gB, = ax.plot([], [], 'o', markersize=1.5, color='darkred')

ax.axis('equal')
ax.set_xlim([-size/2.,size/2.])
ax.set_ylim([-size/2.,size/2.])
ax.set_xlabel('x [kpc]')
ax.set_ylabel('y [kpc]')
#ax.set_facecolor("black")
txt = ax.text(0.4*size,-0.4*size, ' Myr', horizontalalignment='right')

def animate(i):
	gA.set_data(pt[i,2:nA+2,0], pt[i,2:nA+2,1])
	gB.set_data(pt[i,nA+2:,0], pt[i,nA+2:,1])
	txt.set_text('{:.0f} Myr'.format(i*dtout*dt))
	if(fout > 0 and i % fout == 0):
		plt.savefig("freal2_"+str(i).zfill(5)+".png")
	return gA, gB, txt

nf = int(np.ceil(t_end/dt/dtout))
ani = animation.FuncAnimation(fig, animate, frames=nf, interval=animfreq, blit=True)
#ani.save('movie.mp4')
HTML(ani.to_html5_video())
